## HOJA DE TRABAJO 8 - REDES NEURONALES

In [2]:
# Importando librerias necesarias

import matplotlib.pyplot as plt
import numpy as np
import random
import statsmodels.api as sm
import statsmodels.stats.diagnostic as diag
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

#Metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score, f1_score
from sklearn.metrics import make_scorer, accuracy_score, precision_score

In [105]:
# Cargando dataset con pandas
data = pd.read_csv('train.csv')
data = pd.DataFrame(data)


# Limpieza de datos
columns_used = []

data['neighborhood'] = 0
NEIGHBORHOOD = list(set(nh for nh in data['Neighborhood']))
for index, neighborhood_name in enumerate(NEIGHBORHOOD):
    data.loc[data['Neighborhood'] == neighborhood_name, 'neighborhood'] = index + 1
columns_used.append('neighborhood')

data['houseStyle'] = 0
HOUSE_STYLE = list(set(hs for hs in data['HouseStyle']))
for index, houseStyle_name in enumerate(HOUSE_STYLE):
    data.loc[data['HouseStyle'] == houseStyle_name, 'houseStyle'] = index + 1
columns_used.append('houseStyle')

data['houseZone'] = 0
MS_ZONING = list(set(msz for msz in data['MSZoning']))
for index, MSZoning_name in enumerate(MS_ZONING):
    data.loc[data['MSZoning'] == MSZoning_name, 'houseZone'] = index + 1
columns_used.append('houseZone')

data['houseUtilities'] = 0
UTILITIES = list(set(u for u in data['Utilities']))
for index, utils in enumerate(UTILITIES):
    data.loc[data['Utilities'] == utils, 'houseUtilities'] = index + 1
columns_used.append('houseUtilities')

data['roadAccess'] = 0
CONDITION_1 = list(set(c1 for c1 in data['Condition1']))
for index, c1 in enumerate(CONDITION_1):
    data.loc[data['Condition1'] == c1, 'roadAccess'] = index + 1
columns_used.append('roadAccess')

data['remodelated'] = 0
data.loc[data['YearBuilt'] != data['YearRemodAdd'], 'remodelated'] = 1
columns_used.append('remodelated')

data['roofStyle'] = 0
ROOF_STYLE = list(set(rs for rs in data['RoofStyle']))
for index, rs in enumerate(ROOF_STYLE):
    data.loc[data['RoofStyle'] == rs, 'roofStyle'] = index + 1
columns_used.append('roofStyle')

data['roofMaterial'] = 0
ROOF_MATL = list(set(rm for rm in data['RoofMatl']))
for index, rm in enumerate(ROOF_MATL):
    data.loc[data['RoofMatl'] == rm, 'roofMaterial'] = index + 1
columns_used.append('roofMaterial')

data['exteriorCondition'] = 0
EXTER_COND = list(set(ec for ec in data['ExterCond']))
for index, ec in enumerate(EXTER_COND):
    data.loc[data['ExterCond'] == ec, 'exteriorCondition'] = index + 1
columns_used.append('exteriorCondition')

data['foundationMaterial'] = 0
FOUNDATION = list(set(f for f in data['Foundation']))
for index, f in enumerate(FOUNDATION):
    data.loc[data['Foundation'] == f, 'foundationMaterial'] = index + 1
columns_used.append('foundationMaterial')

data['basement'] = 0
data.loc[~data['BsmtQual'].isna(), 'basement'] = 1
columns_used.append('basement')

data['basementCondition'] = 1
data.loc[data['BsmtCond'] == "Ex", 'basementCondition'] = 3
data.loc[data['BsmtCond'] == "Gd", 'basementCondition'] = 2
data.loc[data['BsmtCond'].isna(), 'basementCondition'] = 0
columns_used.append('basementCondition')

data['fireplace'] = 0
data.loc[~data['FireplaceQu'].isna(), 'fireplace'] = 1
columns_used.append('fireplace')

data['pool'] = 0
data.loc[~data['PoolQC'].isna(), 'pool'] = 1
columns_used.append('pool')

data['additionalFeature'] = 0
data.loc[~data['MiscFeature'].isna(), 'additionalFeature'] = 1
columns_used.append('additionalFeature')

data['saleType'] = 0
SALE_TYPE = list(set(st for st in data['SaleType']))
for index, st in enumerate(SALE_TYPE):
    data.loc[data['SaleType'] == st, 'saleType'] = index + 1
columns_used.append('saleType')

data['overallQuality'] = data['OverallQual']
columns_used.append('overallQuality')

data['overallCondition'] = data['OverallCond']
columns_used.append('overallCondition')

data['livingArea'] = data['GrLivArea']
columns_used.append('livingArea')

data['yearBuilt'] = data['YearBuilt']
columns_used.append('yearBuilt')

data['salePrice'] = data['SalePrice']
columns_used.append('salePrice')

columns_not_used = [x for x in data.columns if x not in columns_used]
data = data.drop(columns_not_used, axis=1)

data.head()


,neighborhood,houseStyle,houseZone,houseUtilities,roadAccess,remodelated,roofStyle,roofMaterial,exteriorCondition,foundationMaterial,basement,basementCondition,fireplace,pool,additionalFeature,saleType,overallQuality,overallCondition,livingArea,yearBuilt,salePrice
0,5,4,5,2,3,0,6,2,3,3,1,1,0,0,0,6,7,5,1710,2003,208500
1,19,1,5,2,9,0,6,2,3,1,1,1,1,0,0,6,6,8,1262,1976,181500
2,5,4,5,2,3,1,6,2,3,3,1,1,1,0,0,6,7,5,1786,2001,223500
3,13,4,5,2,3,1,6,2,3,4,1,2,1,0,0,6,7,5,1717,1915,140000
4,18,4,5,2,3,0,6,2,3,3,1,1,1,0,0,6,8,5,2198,2000,250000
5,3,8,5,2,3,1,6,2,3,6,1,1,0,0,1,6,5,5,1362,1993,143000
6,17,1,5,2,3,1,6,2,3,3,1,1,1,0,0,6,8,5,1694,2004,307000
7,1,4,5,2,5,0,6,2,3,1,1,1,1,0,1,6,7,6,2090,1973,200000
8,4,8,1,2,1,1,6,2,3,4,1,1,1,0,0,6,7,5,1774,1931,129900
9,23,7,5,2,1,1,6,2,3,4,1,1,1,0,0,6,5,6,1077,1939,118000


### Conjuntos de entrenamiento y prueba